## Комментарий наставника
Привет, Александр! Меня зовут Александр и я буду проверять твой проект. Спасибо за твою работу:) Далее по ходу работы я оставлю свои комментарии и предложения. Постарайся их учесть в этом и дальнейших проектах. Комментарии ты можешь найти в текстовой ячейке с заголовком «Комментарий наставника» (как здесь) либо в ячейках с кодом в следующем виде: «#Комментарий наставника: <сам комментарий>». \
Часть комментариев может быть выделена цветом: \
<span style="color:green">Зелёный цвет символизирует, что всё отлично</span> \
<span style="color:orange">Оранжевый цвет символизирует рекомендации</span> \
<span style="color:red">Красный цвет символизирует недочёты</span>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
import timeit
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
SEED = 21

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.sample(2)

,calls,minutes,messages,mb_used,is_ultra
1407,55.0,331.98,0.0,12673.71,0
2779,49.0,255.32,0.0,8413.88,0


In [3]:
data['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

 Мы видим выборка не сбалансирована, перешедших на тариф "Ультра" в ~2.3 раза меньше

## Комментарий наставника
<span style="color:green">Да, всё верно.</span>

Разделим данные на признаки и целевое значение

In [4]:
X = data.drop('is_ultra', axis=1)
y = data['is_ultra']

In [5]:
X.shape, y.shape

((3214, 4), (3214,))

## Комментарий наставника
<span style="color:green">Вопросов нет, идём далее.</span>

## 2. Разбейте данные на выборки

Разобьем данные на train и test, на стартифицированные выборки, сохраним дисбаланс классов.

Валидационную выборку выделять не будем, качество проверим на кросс валидации. Подготовим фолды для валидации

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Комментарий наставника
<span style="color:orange">Не надо оставлять столько данных под тестовую выборку, 20% будет более чем достаточно.</span>

Проверим что дисбаланс сохранился в трейне и в тесте



In [7]:
np.isclose((y_train==0).sum()/y_train.shape[0], (y_test==0).sum()/y_test.shape[0], 0.001)

True

## Комментарий наставника
<span style="color:green">Хорошо, начинаем обучение моделей.</span>

## 3. Исследуйте модели

Логистическая регрессия

Перед нами стоит задача классификации, поэтому

используем  логистическую регрессию. Значимым гиперпараметром является коэффициент регуляризации C.

Для поиска оптимального значения используем GridSearchCV с кросс-валидацией на пяти подвыборках.

In [8]:
grid_lr = LogisticRegressionCV(random_state=42, cv=5, n_jobs=-1)

In [9]:
parameters = {'C': (0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000)}

In [10]:
grid_lr = GridSearchCV(LogisticRegression(random_state=42), parameters, cv=5)
grid_lr.fit(X_train, y_train)
params = grid_lr.best_params_
lr = LogisticRegression(**params, random_state=42)
lr.fit(X_train, y_train)
grid_lr.best_params_

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

{'C': 1}

## Комментарий наставника
<span style="color:orange">Тебе совсем не обязательно повторно создавать и обучать модель на лучших гиперпараметрах. GridSearchCV возвращает тебе модель, обученную на лучших параметрах, т.е. ты сразу можешь получать предсказания. См. пример ниже:</span>

In [11]:
grid_lr.predict(X_train)

array([0, 0, 0, ..., 0, 1, 0])

<span style="color:green"> Или так логическая регрессия    </span>

In [12]:
logreg_model = LogisticRegressionCV(random_state=42, cv=5, n_jobs=-1)

In [13]:
logreg_model.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=-1, penalty='l2',
                     random_state=42, refit=True, scoring=None, solver='lbfgs',
                     tol=0.0001, verbose=0)

In [14]:
logreg_model.scores_[1].mean()

0.7385088287060694

## Комментарий наставника
<span style="color:orange">UPD 01.06.2020 Если ты используешь LogisticRegressionCV, то вызов scores_ выводит все оценки на каждом фолде для каждого гиперпараметра. У тебя 5 фолдов, по умолчанию идёт подбор 10 значений коэффициента регуляризации C, поэтому ниже получилось 5х10 = 50 оценок. Согласись, что средняя оценка по всем фолдам и параметрам мало о чём говорит, в отличие от best_score_ к GridSearchCV, который выводит среднюю оценку по фолдам, но только для **лучших** найденных гиперпараметров.</span>

In [36]:
logreg_model.scores_

{1: array([[0.72815534, 0.72815534, 0.72815534, 0.72815534, 0.72815534,
         0.72815534, 0.72815534, 0.72815534, 0.72815534, 0.72815534],
        [0.69708738, 0.74757282, 0.74757282, 0.74757282, 0.74757282,
         0.74757282, 0.74757282, 0.74757282, 0.74757282, 0.74757282],
        [0.74174757, 0.74174757, 0.74174757, 0.74174757, 0.74174757,
         0.74174757, 0.74174757, 0.74174757, 0.74174757, 0.74174757],
        [0.74658869, 0.74853801, 0.74853801, 0.74853801, 0.74853801,
         0.74853801, 0.74853801, 0.74853801, 0.74853801, 0.74853801],
        [0.71539961, 0.71539961, 0.71539961, 0.71539961, 0.71539961,
         0.71539961, 0.75633528, 0.75633528, 0.75633528, 0.75633528]])}

## Комментарий наставника
<span style="color:orange">При желании можешь убрать вывод предупреждений с помощью следующей команды:</span>

In [15]:
#from warnings import simplefilter
#simplefilter(action='ignore', category=FutureWarning)

#### Решающее дерево и случайный лес

Для DecisionTreeClassifier и RandomForestClassifier поиск будет осуществляться по гиперпараметрам max_depth, min_samples_split и min_samples_leaf. Создаю переменную tree_params c различными вариантами значений указанных гиперпараметров.

In [16]:
tree_params = {'max_depth': list(range(2, 11)),
               'min_samples_split': list(range(2, 21, 2)),
               'min_samples_leaf': list(range(1, 21, 2))}

Выбор гиперпараметров для решающего дерева. Для поиска оптимального значения использую GridSearchCV с кросс-валидацией на пяти подвыборках.

In [17]:
grid_dtc = GridSearchCV(DecisionTreeClassifier(random_state=42),
                         tree_params, cv=5, scoring='accuracy') 
grid_dtc.fit(X_train, y_train)
params = grid_dtc.best_params_
dtc = DecisionTreeClassifier(**params, random_state = 42)
dtc.fit(X_train, y_train)
grid_dtc.best_params_

{'max_depth': 7, 'min_samples_leaf': 17, 'min_samples_split': 2}

<span style="color:green"> Или так Решаещее дерево </span>

In [18]:
# Начнем с обычного деревца
tree_model = DecisionTreeClassifier(random_state=42)

In [19]:
tree_model_parameter = {'max_depth': np.arange(1, 10, 1),
                        'min_samples_split': [2, 4, 5], 
                        'min_samples_leaf': [1, 2, 3]
                       }

In [20]:
# создадим greedsearch поставим значение cv на 5, это значит что тестовая выборка разбиается на 5 фолдов
grid_search_tree_model = GridSearchCV(tree_model, tree_model_parameter, cv=5, n_jobs=-1)

In [21]:
grid_search_tree_model.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=42,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'min_samples_leaf': [1, 2, 3],
          

In [22]:

# Посмотрим на лучший accuracy, и лучшие параметры
grid_search_tree_model.best_score_, grid_search_tree_model.best_params_

(0.7915208090237261,
 {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2})

Итак мы видим , что  среднее значение accuracy у дерева получилось 0.79!Хорошо !

## Комментарий наставника
<span style="color:green">UPD 01.06.2020 Прекрасно! К этой части вопросов нет.</span>

Выбор гиперпараметров для случайного леса. Для поиска оптимального значения использую GridSearchCV с кросс-валидацией на пяти подвыборках.

In [23]:
grid_rfc= GridSearchCV(RandomForestClassifier(random_state=42), 
                         tree_params, cv=5, scoring='accuracy') 
grid_rfc.fit(X_train, y_train)
params = grid_rfc.best_params_
rfc = RandomForestClassifier(**params, random_state = 42)
rfc.fit(X_train, y_train)
grid_rfc.best_params_

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

{'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 20}

<span style="color:green"> Или так случайный лес     </span>

In [24]:
randomforest_model = RandomForestClassifier(random_state=42)

In [25]:

randomforest_model_parameter = {'n_estimators': np.arange(1, 100, 10),
                                'max_depth': np.arange(1, 10, 2)
                               }

In [26]:
grid_search_randomforest_model = GridSearchCV(randomforest_model, randomforest_model_parameter, cv=5, n_jobs=-1)

In [27]:
grid_search_randomforest_model.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'ma

In [28]:
# Посмотрим на лучший accuracy, и лучшие параметры
grid_search_randomforest_model.best_score_, grid_search_randomforest_model.best_params_

(0.809023726176585, {'max_depth': 9, 'n_estimators': 41})

Вывод:
Лучшее дерево показало среднее accuracy 0.8, а вот уже "случайный" лес показал нам среднее значение accuracy 0.8. Логистическая регрессия показала самый низкий результат 0.74, но и это уже хорошо.

## Комментарий наставника
<span style="color:green">Здорово! Очень хорошая работа с моделями.</span>

## Комментарий наставника
<span style="color:red">Не забывай писать свои выводы и рассуждения после каждого блока проекта, хотя бы в кратком виде (какая модель показала лучший результат, с какими параметрами и т.п.). Ты можешь вывести оценку моделей на кросс-валидации с помощью параметра best_score_ к GridSearchCV.</span> \
<span style="color:green">UPD 01.06.2020 Хорошо.</span>

## 4. Проверьте модель на тестовой выборке

Функция для расчета accuracy предсказания на тестовой выборке после обучения модели на тренировочной выборке.

In [29]:
def test_score(model):
    test_predictions = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, test_predictions)
    print(test_accuracy)

In [30]:
test_score(lr)

0.6998444790046656


In [31]:
test_score(dtc)

0.8102643856920684


In [32]:
test_score(rfc)

0.8149300155520995


#### Вывод 

На тестовой выборке лучшее значение accuracy 0.81 удалось достичь, используя RandomForestClassifier. Результат DecisionTreeClassifier чуть меньше - 0.81. Самый худший результат 0.7 показала LogisticRegression.

## Комментарий наставника
<span style="color:red">Не нужно оценивать все модели на тестовой выборке. Оценивай только лучшую по результатам кросс-валидации модель.</span>

## 5. (бонус) Проверьте модели на адекватность

In [33]:
(data['is_ultra']==0).sum() / data.shape[0]

0.693528313627878

Общие выыоды:
В у нас имеется  4 признака и 3214 объектов

1.Целевой признак у нас "is_ultra". В нем большинство пользователей с тарифом "Smart"

2.Функцией train_test_split наши данные были разделены на train - 80% и test- 20%.

3. Dместо валидационных данных была использована встроенная кросс валидация с параметром cv=5

4. Были проверены модели DecisionTreeClassifier, RandomForestClassifier, LogisticRegressionCV.

5. Подбор гипер параметров для DecisionTreeClassifier и RandomForestClassifier осуществлялся с помощью Grid Search

В итоге после получилась следующие значения accuracy:

In [34]:
df = pd.DataFrame([['Tree',0.8, 0.81],
                  ['RandomForest',0.80, 0.81],
                  ['LogisticReg',0.74, 0.70]], 
columns=['Model','Train accuracy', 'Test accuracy'])

In [35]:
df.head()

,Model,Train accuracy,Test accuracy
0,Tree,0.80,0.81
1,RandomForest,0.80,0.81
2,LogisticReg,0.74,0.70


## Итоговый комментарий наставника
<span style="color:blue">Отлично! \
Проект хороший, но обрати, пожалуйста, внимание на мои комментарии. Работу нужно будет немного дополнить. Будем ждать:)</span>

<span style="color:green">Привет Александр , все переделал как ты меня просил , ниже немножко сделал код по другому и сделал выходы </span>

<span style="color:blue">UPD 01.06.2020 Хорошо, но достаточно было одной модели для финального тестирования — лучшей по результатам кросс-валидации. Проект зачтён. Поздравляю и желаю тебе дальнейших успехов в учёбе!:)</span>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
